### Import the libraries

In [8]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Load the dataset

In [9]:
mnist_dataset, mnist_info=tfds.load(name='mnist', as_supervised=True, with_info=True) # As supervised loads the data in a two
# tuple structure(inputs and targets). With info provides a tuple containing info about version, features and 
# the number of samples in the dataset

### Preprocess the Data

In [10]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test'] # We define the train and test datasets. 
# The MNIST dataset does not come with validation samples so we have to do a split. 

# We take 10 percent of the training dataset as our validation samples
mnist_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

# The computation above could result in a number of samples with decimal places which isn't ideal, so we cast the data
mnist_validation_samples = tf.cast(mnist_validation_samples, tf.int64)

# We do the same for the test data as we would need this later during batching
mnist_test_samples = mnist_info.splits['test'].num_examples
mnist_test_samples = tf.cast(mnist_test_samples, tf.int64)

# To ensure our result is numerically stable, we scale the data set to have inputs between 0 and 1

def scale(image, label):
    image = tf.cast(image, tf.float32) # To ensure the values are in float as we will be scaling
    image /= 255.0 # Dividing by 255 will keep all values between 0 and 1
    return image, label

scaled_train_validation = mnist_train.map(scale) # This transforms the dataset using the scale function created above
scaled_test_data = mnist_test.map(scale) # Same process for the test dataset as we want the test data to have the 
# same magnitude as that of the training and validation data

# Now we shuffle the dataset

BUFFER_SIZE = 10000 # We create a buffer size in order to optimize the computer's memory. This buffer will be used to
# shuffle 10000 samples per time

shuffled_scaled_train_validation = scaled_train_validation.shuffle(BUFFER_SIZE)

# We can now split the dataset into training and validation dataset

validation_data = shuffled_scaled_train_validation.take(mnist_validation_samples) # Takes 10 percent of the shuffled
# data as the validation data
training_data = shuffled_scaled_train_validation.skip(mnist_validation_samples) # Skip as many samples as we have in
# our validation data

# We apply a mini-batch gradient descent with a batch size of 100
# The validation data doesn't need to be batched as we don't need to update the weights and biases(backward propagation)
# The inputs from the validation data are only pushed through the net(forward propagation), hence we do create mini
# batches for our validation data

BATCH_SIZE = 100
training_data = training_data.batch(BATCH_SIZE)


# Since our model expects our validation and test data in batch form too, we also batch the data but all the data goes in 
# a single batch

validation_data = validation_data.batch(mnist_validation_samples)
test_data = scaled_test_data.batch(mnist_test_samples) # Test data also goes in a single batch

# We have a two tuple structure with inputs and targets. Recall enabling as_supervised. We then create an iterable
# and load the next batch(there is only one batch)
validation_inputs, validation_targets = next(iter(validation_data))

### Outline the Model

In [11]:
# First,we define the hyperparameters

input_size = 784
output_size = 10
hidden_layer_size = 500 # Assuming all hidden layers are of the same width

# Since our input is a tensor of rank 3 (28x28x1), we flatten the input into a vector to feed into the network
# We then build a net with 2 hidden layers with the activation function as ReLu and the output unit using the 
# softmax activation function in order to generate a probabilistic output.
model = tf.keras.Sequential([
    
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Select optimizers and loss function 

In [12]:
# We use the Adaptive moment estimation as our optimizer. The loss function here is selected in order to pass in 
# integer inputs and transform into one-hot encoding. In addition, we add the accuracy as part of the metrics to 
# track during the training of our model
#custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.02)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'],)

### Training

In [14]:
# We define the number of epochs
NUM_EPOCHS=10

model.fit(training_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets),
          validation_steps=1, verbose=2)

Epoch 1/10
540/540 - 9s - loss: 0.0092 - accuracy: 0.9974 - val_loss: 0.0092 - val_accuracy: 0.9970
Epoch 2/10
540/540 - 9s - loss: 0.0134 - accuracy: 0.9955 - val_loss: 0.0366 - val_accuracy: 0.9883
Epoch 3/10
540/540 - 9s - loss: 0.0127 - accuracy: 0.9962 - val_loss: 0.0144 - val_accuracy: 0.9950
Epoch 4/10
540/540 - 9s - loss: 0.0117 - accuracy: 0.9957 - val_loss: 0.0165 - val_accuracy: 0.9938
Epoch 5/10
540/540 - 9s - loss: 0.0116 - accuracy: 0.9959 - val_loss: 0.0122 - val_accuracy: 0.9960
Epoch 6/10
540/540 - 9s - loss: 0.0089 - accuracy: 0.9971 - val_loss: 0.0143 - val_accuracy: 0.9955
Epoch 7/10
540/540 - 9s - loss: 0.0107 - accuracy: 0.9965 - val_loss: 0.0162 - val_accuracy: 0.9950
Epoch 8/10
540/540 - 9s - loss: 0.0094 - accuracy: 0.9971 - val_loss: 0.0117 - val_accuracy: 0.9965
Epoch 9/10
540/540 - 10s - loss: 0.0094 - accuracy: 0.9973 - val_loss: 0.0055 - val_accuracy: 0.9983
Epoch 10/10
540/540 - 11s - loss: 0.0055 - accuracy: 0.9982 - val_loss: 0.0098 - val_accuracy: 0.99

### Test the model

In [15]:
# We push the test data through the network(forward propagation) in order to obtain the accuracy of the model
# The test accuracy tells us the accuracy to expect when we deploy this model in the real world
test_loss, test_accuracy = model.evaluate(test_data) 

      1/Unknown - 1s 1s/step - loss: 0.1080 - accuracy: 0.9805

In [16]:
print("Test Loss : {0:.2f}, Accuracy : {1:.2f}%".format(test_loss, test_accuracy*100.0))

Test Loss : 0.11, Accuracy : 98.05%
